## Importación de bibliotecas 

In [1]:
# bibliotecas (libraries) incorporadas (built-in) de Python
import os
from random import randint
# Para cargar solo una vez las bibliotecas detecta si es la primer ejecución del cuaderno
try:
    os.getegid()
    firstrun = False
except:
    firstrun = True

In [2]:
# Entorno de ejecución

# if (firstrun):
if('google.colab' in str(get_ipython() ) ):
    environment= 'google'
else:
    import os
    if (os.environ.get('PWD')=='/kaggle/working'):
        environment= 'kaggle'
    else:
        environment= 'local'
print(environment)

local


In [3]:
if (environment =='local'):
  try:
    import tensorrt
  except:
    ! pip install tensorrt
    import tensorrt

ModuleNotFoundError: No module named 'tensorrt_bindings'

In [8]:
if (firstrun):
    try:
        from tensorflow.keras.utils  import load_img
    except:
        ! pip install tensorflow
        from tensorflow.keras.utils  import load_img

# from keras.preprocessing.image import load_img
from tensorflow.keras.utils  import load_img
#from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import img_to_array

ModuleNotFoundError: No module named 'tensorrt_bindings'

¿Por qué `tensorflow.keras` y `keras`?  
[Stack overflow | keras vs. tensorflow.python.keras - which one to use?](https://stackoverflow.com/questions/48893528/keras-vs-tensorflow-python-keras-which-one-to-use)
> tensorflow.python.keras is just a bundle of keras with a single backend inside tensorflow package. This allows you to start using keras by installing just pip install tensorflow.
> 
> keras package contains full keras library with three supported backends: tensorflow, theano and CNTK. If you even wish to switch between backends, you should choose keras package. This approach is also more flexible because it allows to install keras updates independently from tensorflow (which may not be easy to update, for example, because the next version may require a different version of CUDA driver) or vice versa. For this reason, I prefer to install keras as another package.

## Carga de muestra de imágenes

In [5]:
# Directorio base ( cambiar según el sistema de archivos de cada uno)
# if (firstrun):
if( environment== 'local' ):
    # system_path = "C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMCyT/TP/"
    system_path = '/home/vbettachini/documents/universitet/FCEyN/maestríaDatos/cienciaTecnología/'
elif( ( environment== 'google' ) ): 
    from google.colab import drive
    drive.mount('/content/drive')
    system_path = "/content/drive/MyDrive/maestría/cienciaTecnología/"
elif( ( environment== 'kaggle' ) )  :
    a= 1

# Directorio del dataset
dataset_path = system_path + "tp1_dmcyt2023/datasets/Rice_Image_Dataset"

In [7]:
# Path a subset imágenes generado por `random_sample_rice_images.ipynb`
image_path = dataset_path + "/random_sample_rice_images"

In [8]:
path = sorted(
  [
    os.path.join(image_path, file)
    for file in os.listdir(image_path )
    if file.endswith('.jpg')
  ]
)
print(len(path))
print(path[0])

5000
/home/vbettachini/documents/universitet/FCEyN/maestríaDatos/cienciaTecnología/tp1_dmcyt2023/datasets/Rice_Image_Dataset/random_sample_rice_images/Arborio (1002).jpg


## Generación de features

In [9]:
def extract_features(file, model):
    # levanta imagen como array 224x224
    img = load_img(file, target_size=(224,224))
    # convierte img a numpy array (originalmente es 'PIL.Image.Image')
    img = np.array(img)
    # reshape para tener formato necesario para el modelo (num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3)
    # prepara imagen para modelo (función de keras)
    imgx = preprocess_input(reshaped_img)
    # extrae features
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [10]:
def preprocess(lista, model):
    datays = {}
    for i in lista:
      featuress = extract_features(i, model)
      datays[i] = featuress
    # lista de filenames
    filenmss = np.array(list(datays.keys()))

    # lista de features
    featsss = np.array(list(datays.values()))

    # reshape 4096 features por el número de imagens
    print(featsss.shape)
    featsss = featsss.reshape(-1,4096)
    return filenmss, featsss

In [11]:
if (firstrun):
    try:
        from keras.models import Model
    except:
        ! pip install keras
        from keras.models import Model

from keras.models import Model
# Tensorflow model
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

2023-09-27 12:26:27.189892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 12:26:27.571672: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-09-27 12:26:28.011507: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-09-27 12:26:28.530439: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free s

In [12]:
if (firstrun):
  try:
    import numpy as np
  except:
    ! pip install numpy
    import numpy as np

import numpy as np

In [13]:
## Levanta archivos y extrae features
names, feat = preprocess(path, model)

1/1 [==============================] - 1s 1s/step


In [ ]:
np.savez_compressed(image_path + '/vgg16output.npz', feat = feat, names = names)